In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers, ctransformers
from langchain.chains import RetrievalQA

import chainlit as cl

from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [2]:
!nvidia-smi

Sun Jul 30 23:00:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:01:00.0  On |                  N/A |
| 33%   32C    P8              17W / 350W |    576MiB / 24576MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# pip install langchain
# import langchain
langchain.__version__

'0.0.247'

In [2]:
# pip install chainlit
cl.__version__

'0.6.1'

In [10]:
# pip install sentence_transformers

In [4]:
# pip install pypdf
import pypdf
pypdf.__version__

'3.14.0'

In [7]:
# pip install faiss-cpu
# import faiss-cpu
# faiss-cpu == 1.7.4


                                              0.0/10.8 MB ? eta -:--:--
                                              0.0/10.8 MB 1.9 MB/s eta 0:00:06
                                              0.1/10.8 MB 1.1 MB/s eta 0:00:10
                                             0.1/10.8 MB 871.5 kB/s eta 0:00:13
                                             0.1/10.8 MB 819.2 kB/s eta 0:00:14
                                             0.1/10.8 MB 654.9 kB/s eta 0:00:17
                                             0.2/10.8 MB 653.6 kB/s eta 0:00:17
                                             0.2/10.8 MB 655.1 kB/s eta 0:00:17
                                             0.2/10.8 MB 715.9 kB/s eta 0:00:15
     -                                       0.3/10.8 MB 737.3 kB/s eta 0:00:15
     -                                       0.3/10.8 MB 752.2 kB/s eta 0:00:14
     -                                       0.4/10.8 MB 764.2 kB/s eta 0:00:14
     -                                       0.4/

In [28]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [29]:
import transformers
transformers.__version__

'4.31.0'

In [10]:
pip install ctransformers

                                              0.0/6.5 MB ? eta -:--:--
                                              0.0/6.5 MB ? eta -:--:--
                                              0.1/6.5 MB 2.0 MB/s eta 0:00:04
                                              0.1/6.5 MB 1.2 MB/s eta 0:00:06
     -                                        0.2/6.5 MB 1.3 MB/s eta 0:00:05
     -                                        0.2/6.5 MB 1.3 MB/s eta 0:00:06
     -                                        0.3/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.3/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.4/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.4/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.5/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.5/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.6/6.5 MB 1.2 MB/s eta 0:00:05


In [12]:
# import ctransformers
# ctransformers.__version__

AttributeError: module 'ctransformers' has no attribute '__version__'

In [4]:
# !nvidia-smi

In [3]:
DATA_PATH = 'data/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [3]:


# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#                                        ,model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

# create_vector_db()

2023-07-30 22:41:48 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-07-30 22:41:49 - Use pytorch device: cpu


Batches:   0%|          | 0/224 [00:00<?, ?it/s]

2023-07-30 22:46:08 - Loading faiss with AVX2 support.
2023-07-30 22:46:08 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-07-30 22:46:08 - Loading faiss.
2023-07-30 22:46:08 - Successfully loaded faiss.


In [8]:
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
#                                        model_kwargs={'device': 'cpu'})

2023-07-30 21:41:21 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [4]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [5]:
#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 4}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [6]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

In [7]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [8]:
#output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [9]:
query = "My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt()
qa_result = retrieval_qa_chain(llm, qa_prompt, db)
response = qa_result({'query': query})
response

2023-07-30 23:00:45 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-07-30 23:00:46 - Loading faiss with AVX2 support.
2023-07-30 23:00:46 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-07-30 23:00:46 - Loading faiss.
2023-07-30 23:00:46 - Successfully loaded faiss.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?',
 'result': "Antidiarrested around or a combination of information is caused by the user578 hours after taking certain antidiarrestabdomesticated with diarrhese drugs may not always consult with acute and irriting and also occurs when you don'\nI cannot be caused by taking medication,\nAntidiarrestabdomestic ulcervom/p> Antidiarrestabdominate.\nIf your symptoms the doctorVisiting abdominantly with lactually becomes more than two days or above the symptoms are there are you don'I don'The information about \n• Diarrestabdom, diarrested to diagnose. I'You'Hi!\nSorry, Yes, you don'\nAntidiarrhese drugs\nDiarrestabdomic diarrested at first of  Based on Antidiopin a common causes of the symptomsom/p> Diarrestabdomestic ulcervomes not enough, thank you could you need to prevent diarrhesitation to aThe ter

In [12]:
response['result']

"Antidiarrested around or a combination of information is caused by the user578 hours after taking certain antidiarrestabdomesticated with diarrhese drugs may not always consult with acute and irriting and also occurs when you don'\nI cannot be caused by taking medication,\nAntidiarrestabdomestic ulcervom/p> Antidiarrestabdominate.\nIf your symptoms the doctorVisiting abdominantly with lactually becomes more than two days or above the symptoms are there are you don'I don'The information about \n• Diarrestabdom, diarrested to diagnose. I'You'Hi!\nSorry, Yes, you don'\nAntidiarrhese drugs\nDiarrestabdomic diarrested at first of  Based on Antidiopin a common causes of the symptomsom/p> Diarrestabdomestic ulcervomes not enough, thank you could you need to prevent diarrhesitation to aThe term usually indicates that lastsometimes it is there are there are there are there are there are there are there are there are some types of the most likely caused"

In [10]:
!nvidia-smi

Sun Jul 30 23:02:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:01:00.0  On |                  N/A |
| 33%   31C    P8              16W / 350W |    579MiB / 24576MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--